In [1]:
import sys
sys.path.append("/disk/xray15/aem2/camels/proj2")
from setup_params_alice import process_all_variations, process_colours_all, process_lfs_all, plot_single_measurement

from variables_config_28 import (
    param_info_file, 
    plots_dir, 
    redshift_values, 
    lf_data_dir,
    colour_data_dir, 
    uvlf_limits, 
    colour_limits,
    filters,
    colour_pairs
    
)

In [2]:
print("Checking variables:")
print(f"param_info_file: {param_info_file}")
print(f"filters: {filters}")
print(f"redshift_values: {redshift_values}")
print(f"lf_data_dir: {lf_data_dir}")
print(f"plots_dir: {plots_dir}")


Checking variables:
param_info_file: /disk/xray15/aem2/data/28pams/Info_IllustrisTNG_L25n256_28params.txt
filters: {'intrinsic': ['UV1500', 'GALEX FUV', 'GALEX NUV'], 'attenuated': ['GALEX FUV', 'GALEX NUV']}
redshift_values: {'044': {'redshift': 2.0, 'label': 'z2.0'}, '052': {'redshift': 1.48, 'label': 'z1.5'}, '060': {'redshift': 1.05, 'label': 'z1.0'}, '086': {'redshift': 0.1, 'label': 'z0.1'}}
lf_data_dir: {'attenuated': {'GALEX': '/disk/xray15/aem2/data/28pams/IllustrisTNG/1P/LFs/attenuated/GALEX'}, 'intrinsic': {'UV1500': '/disk/xray15/aem2/data/28pams/IllustrisTNG/1P/LFs/intrinsic/UV1500', 'GALEX': '/disk/xray15/aem2/data/28pams/IllustrisTNG/1P/LFs/intrinsic/GALEX'}}
plots_dir: /disk/xray15/aem2/plots/28pams/IllustrisTNG/1P


In [ ]:

if __name__ == "__main__":
    process_all_variations(
        # plots redshift variation and parameter variation for each parameter.
        param_info_file=param_info_file,
        filters=filters,
        redshift_values=redshift_values,
        lf_data_dir=lf_data_dir,
        plots_dir=plots_dir
)    

In [3]:

if __name__ == "__main__":
    # creates all colours in one pdf showing all 28
    process_lfs_all(redshift_values, filters, param_info_file, lf_data_dir, plots_dir)
    #process_colours_all(redshift_values, colour_pairs, param_info_file,colour_data_dir,colour_limits,plots_dir)

Available redshifts: dict_keys(['044', '052', '060', '086'])

Processing redshift 2.0 (snap 044)
Processing band type: intrinsic
Bands to process: ['UV1500', 'GALEX FUV', 'GALEX NUV']
Creating UVLF parameter grid for intrinsic UV1500

DEBUG: Starting create_lf_all
band_type: intrinsic
band_or_colour_pairs: UV1500

Parameter 1, Variation n2:
Magnitude range: [-26.61, -16.39]
Phi range: [-6.00, -3.25]
Line added to plot: [<matplotlib.lines.Line2D object at 0x7f6dbbca3250>]

Parameter 1, Variation n1:
Magnitude range: [-26.61, -16.39]
Phi range: [-6.00, -2.73]
Line added to plot: [<matplotlib.lines.Line2D object at 0x7f6dbbca37f0>]

Parameter 1, Variation 1:
Magnitude range: [-26.61, -16.39]
Phi range: [-6.00, -2.47]
Line added to plot: [<matplotlib.lines.Line2D object at 0x7f6dbbca3bb0>]

Parameter 1, Variation 2:
Magnitude range: [-26.61, -16.39]
Phi range: [-5.97, -2.34]
Line added to plot: [<matplotlib.lines.Line2D object at 0x7f6dbbca3d60>]
Setting up panel 1 with 4 plots

Parameter 

In [ ]:

# plot single file
plot_single_measurement(
    "/disk/xray15/aem2/data/28pams/IllustrisTNG/1P/LFs/intrinsic/UV1500/z0.1/UVLF_1P_p1_0_UV1500_z0.1_intrinsic.txt",
    "/disk/xray15/aem2/data/28pams/IllustrisTNG/1P/colours/intrinsic/GALEX_FUV-NUV/z0.1/Colour_1P_p1_0_GALEX_FUV-NUV_z0.1_intrinsic.txt",
    band_info=("GALEX FUV", "2.0", "intrinsic"),
    color_info=("GALEX FUV", "GALEX NUV", "2.0", "intrinsic"),
    output_dir="/disk/xray15/aem2/plots/28pams/IllustrisTNG/1P/test"
)
